In [59]:
import MySQLdb
import simplejson as json
import requests
from bs4 import BeautifulSoup
import re

db = MySQLdb.connect(host="localhost", port=3306,
                     user="root", 
                      passwd="mysql", 
                      db="civic")

url = "http://myneta.info/delhi2015/index.php?action=show_winners&sort=default"
politician = [None] * 7
headers = {'Accept': 'text/html'}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content)
tables = soup.findAll("table")
for idx, table in enumerate(tables):
    if(idx == 2):
        allrows = table.find_all('tr')
        del allrows[0]
        del allrows[0]
        for row in allrows:
            cells = row.find_all("td")
            del cells[0]
            for colIndex, cell in enumerate(cells):
                if(colIndex == 0):
                    for anchorIndex, a in enumerate(cell.findAll('a')):
                        if(anchorIndex == 1):
                            politician[0] = a.contents[0]
                            break
                if(colIndex == 1):
                    cur = db.cursor() 
                    cur.execute("select id from c_constituency where name='"+cell.contents[0]+"'");
                    for row in cur.fetchall() :
                        politician[1] = row[0]
                if(colIndex == 2):
                    cur = db.cursor() 
                    cur.execute("select id from c_party where abbreviation='"+cell.contents[0]+"'");
                    for row in cur.fetchall() :
                        politician[2] = row[0]
                if(colIndex == 3):
                    for spanIndex, b in enumerate(cell.findAll('span')):
                        politician[3] = b.contents[0]
                        break
                    if (not cell.findAll('span')):
                        politician[3] = cell.contents[0]
                if(colIndex == 4):
                    politician[4] = cell.contents[0]
                if(colIndex == 5):
                    politician[5] = re.sub("[^0-9]", "", cell.contents[0])
                if(colIndex == 6):
                    politician[6] = re.sub("[^0-9]", "", cell.contents[0])
            cur = db.cursor() 
            query = 'insert into c_politician(name,constituency_id,party_id,education,totalAssets,liabilities,criminal_cases_pending) values ("'
            query = query + politician[0]+'",'
            query = query + str(politician[1])+','
            query = query + str(politician[2])+',"'
            query = query + politician[4]+'",'
            query = query + str(politician[5])+','
            query = query + str(politician[6])+','
            query = query + str(politician[3])+')'
            cur.execute(query)
            print query, cur.rowcount
            db.commit()
            cur.close ()
        db.close ()
    

insert into c_politician(name,constituency_id,party_id,education,totalAssets,liabilities,criminal_cases_pending) values ("Pawan Kumar Sharma",1,1,"12th Pass",10996895,256710, 1 ) 1
insert into c_politician(name,constituency_id,party_id,education,totalAssets,liabilities,criminal_cases_pending) values ("Ajay Dutt",2,1,"Post Graduate",23427501,2972000,0) 1
insert into c_politician(name,constituency_id,party_id,education,totalAssets,liabilities,criminal_cases_pending) values ("Gopal Rai",3,1,"Post Graduate",5239843,0, 2 ) 1
insert into c_politician(name,constituency_id,party_id,education,totalAssets,liabilities,criminal_cases_pending) values ("Narayan Dutt Sharma",4,1,"12th Pass",6178144,50000,0) 1
insert into c_politician(name,constituency_id,party_id,education,totalAssets,liabilities,criminal_cases_pending) values ("Ajesh Yadav",5,1,"Graduate",145047653,15404944,0) 1
insert into c_politician(name,constituency_id,party_id,education,totalAssets,liabilities,criminal_cases_pending) values ("